# Meteo con el pato

### Este informe muestra los datos diarios de temperatura recogidos en el portal de datos abiertos del ayuntamiento.
* dataset: **meteorologia-tiempo-real** 

In [ ]:
import duckdb
import plotly.express as px

%load_ext sql

In [50]:
%%capture

#conexion a bbdd en memoria o en fichero
#conn = duckdb.connect() 
conn = duckdb.connect('/home/jcaubin/duck_test.db') 


%sql conn --alias duckdb
#limite de filas a mostrar
%config SqlMagic.displaylimit = 100 

In [ ]:
# imprime la hora de ejecución
from datetime import datetime
from IPython.display import display, Markdown

t = datetime.now()
display(Markdown(f"### hora de ejecución: {t}"))

## METEO

In [52]:

magnitud = 83 #temperatura

In [ ]:
%%sql 
result2 << 
select estacion_desc,dia,  h, valor 
from CALAIR 
where magnitud = {{magnitud}} 
and validez = 'V'
and estacion_desc in ['Casa de Campo'] --, 'Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
order by estacion_desc, h


In [54]:
df = result2.DataFrame()

In [ ]:
import plotly.express as px

#df = result2.DataFrame()

fig = px.line(df, x="H", y="VALOR", color="DIA", symbol='ESTACION_DESC', title='Temperaturas hoy')
fig.show()

In [ ]:
%%sql
--tabla ancha
PIVOT(
select estacion_desc, h, valor
from CALAIR
where
magnitud = {{magnitud}} 
and validez = 'V'
and DIA = datepart('day',current_date())
)
on ESTACION_DESC
USING sum(VALOR)
group by H

In [ ]:
%%sql 
precipitaciones << 
select estacion_desc,dia,  h, valor 
from CALAIR 
where magnitud = 89
and validez = 'V'
and estacion_desc in ['Casa de Campo'] --, 'Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
order by estacion_desc, h


In [ ]:
fig = px.line(precipitaciones.DataFrame(), x="H", y="VALOR", color="DIA", symbol='ESTACION_DESC', title='Precipitaciones ')
fig.show()

In [ ]:
%%sql 
dm <<
select estacion_desc, parametro,dia,  TS, valor 
from v_CALAIR 
where  date_diff ('hour',TS, current_date()) < 48 --DIA = datepart('day',current_date())
and validez = 'V'
and magnitud >=80
--and estacion_desc in ['Casa de Campo','Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
order by estacion_desc, TS

In [ ]:
fig = px.line(dm.DataFrame(), x="TS", y="VALOR", color="ESTACION_DESC", facet_col='PARAMETRO',facet_col_wrap=2 ,title='Meteorología ', height=2000)
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig.for_each_xaxis(lambda a: a.update(showticklabels=True))
#fig.update_layout(xaxis=dict(range=[0,24]))
#fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.show()

## CALIDAD DEL AIRE

In [ ]:
%%sql 
nox << 
select estacion_desc,dia,  h, valor 
from CALAIR 
where magnitud = 08
and validez = 'V'
--and estacion_desc in ['Casa de Campo','Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
order by estacion_desc, h

In [ ]:
fig = px.line(nox.DataFrame(), x="H", y="VALOR", color="DIA", symbol='ESTACION_DESC', title='NO2 ')
fig.show()

In [ ]:
fig = px.line(nox.DataFrame(), x="H", y="VALOR", color="DIA", facet_col='ESTACION_DESC',facet_col_wrap=4 ,title='NOX ')
fig.show()

In [ ]:
%%sql
--tabla ancha
PIVOT(
select estacion_desc, h, valor
from calair
where
magnitud = 08
and validez = 'V'
and DIA = datepart('day',current_date())
)
on ESTACION_DESC
USING sum(VALOR)
group by H

In [ ]:
%%sql 
cont <<
select estacion_desc, parametro,dia,  TS, valor 
from v_CALAIR 
where  date_diff ('hour',TS, current_date()) < 48 --DIA = datepart('day',current_date())
and validez = 'V'
and magnitud <80
--and estacion_desc in ['Casa de Campo','Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
order by estacion_desc, TS


In [ ]:
fig = px.line(cont.DataFrame(), x="TS", y="VALOR", color="ESTACION_DESC", facet_col='PARAMETRO',facet_col_wrap=2 ,title='Contaminantes ', height=2000)
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig.for_each_xaxis(lambda a: a.update(showticklabels=True))
#fig.update_layout(xaxis=dict(range=[0,24]))
#fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.show()

In [48]:
%sql --close duckdb